In [3]:
import csv
import sys
import os
import time
import pandas as pd
from datetime import datetime,date
import re
from selenium import webdriver
import requests
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
import pyodbc
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.service import Service
from selenium import webdriver

In [33]:
def date_con(review_date_str):
    pattern = r'(\d+) (\w+)\ ago'
    match = re.match(pattern, review_date_str)
    if match:
        num = int(match.group(1))
        unit = match.group(2)
        units_mapping = {
            'mo': 'months',
            'days': 'days',
            'yr': 'years'
            }
        unit = units_mapping.get(unit, '')
        actual_date = datetime.now() - relativedelta(**{unit: num})
        return actual_date.strftime('%Y-%m-%d')
    else:
        return None

def replace_str(text):
    pattern = r'\bEdited\b(?:\s+\w+){0,2}\s*'
    new_text = re.sub(pattern, '', text)
    return new_text
driver=webdriver.Edge()

# edge_driver_path = r'C:\Users\v-smanish\Downloads\edgedriver_win64\msedgedriver.exe'
# edge_service = Service(edge_driver_path)
# driver = webdriver.Edge(service=edge_service)
urls = {
'Surface':'https://www.reddit.com/r/Surface/search/?q=surface+laptops&type=link&cId=dc438b78-f50d-4cbd-b272-5afe8e3d7409&iId=d569288b-f614-4cd9-8717-e107a9b99beb',
'Surface':'https://www.reddit.com/r/Surface/search/?q=windows+laptop&type=link&cId=f8660466-e8e7-4c67-a88d-11c3e76f8df7&iId=f31227e2-8738-403d-a677-6a44bb445644',
'Laptops':'https://www.reddit.com/r/laptops/search/?q=surface+laptop&type=link&cId=5ad70ba2-3940-4326-9c65-d14485443c5e&iId=2acc74cc-2dfb-4040-b9f9-9f69e7f8be20',
'Laptops':'https://www.reddit.com/r/laptops/search/?q=windows+laptop&type=link&cId=bdbb86cf-75f1-4cb0-af2f-c4e958caea27&iId=b8a04d5c-a047-448b-ad81-6447a2d44770',
'SuggestALaptop':'https://www.reddit.com/r/SuggestALaptop/search/?q=surface+laptop&type=link&cId=384461d3-f65d-4b7f-8180-5e98e33681c1&iId=7c787863-b632-47ea-a51f-52ab5da3c248',
'SuggestALaptop':'https://www.reddit.com/r/SuggestALaptop/search/?q=windows+laptop&type=link&cId=fabbf3ea-bd1f-4798-a897-7514a2d1a1a6&iId=474cd374-8cfe-432b-a1cf-fb761a395755',
'LaptopDeals':'https://www.reddit.com/r/LaptopDeals/search/?q=surface+laptop&type=link&cId=75ca11b7-1f45-438b-80f3-3df017697ee6&iId=e9cddb4d-39e9-49c0-9a42-de41a7f7ea48',
'LaptopDeals':'https://www.reddit.com/r/LaptopDeals/search/?q=windows+laptop&type=link&cId=d7f023ed-c2f5-4055-aa0c-d02678930199&iId=89acd1f1-86ff-4dae-964a-6e441e28488f',
'HardwareSwap':'https://www.reddit.com/r/hardwareswap/search/?q=surface+laptop&type=link&cId=10c6e627-082f-42fc-8475-986698b81214&iId=77c3e153-16c7-4d53-aff0-8a5b43199709',
'HardwareSwap':'https://www.reddit.com/r/hardwareswap/search/?q=windows+laptop&type=link&cId=68f21823-0d52-4dd7-8bc9-e0b532835c73&iId=daa96cf6-70c0-4887-9dd5-3ccee4e99a46',
'Microsoft':'https://www.reddit.com/r/microsoft/search/?q=surface+laptop&type=link&cId=14dedb92-1554-407e-88f1-04b10e2bc837&iId=7e30b003-da3e-445e-b485-39646e56ac26',
'Microsoft':'https://www.reddit.com/r/microsoft/search/?q=windows+laptop&type=link&cId=bb5ef5e7-65d6-484c-9be9-13441fd01a0c&iId=c5e9e960-ceb1-4412-9fd8-836ee54ff0d7',
'AMD':'https://www.reddit.com/r/Amd/search/?q=surface+laptop&type=link&cId=4641767a-5c25-405d-8749-fa8d319cee11&iId=e8d3b0be-e8cb-441e-b75c-23d12bce7cd3',
'AMD':'https://www.reddit.com/r/Amd/search/?q=Windows+laptop&type=link&cId=edda8380-418d-4943-b718-b4fb8b6abaf6&iId=0345e556-d60d-4950-858c-0d9335fd9379',
'Gadget':'https://www.reddit.com/r/gadgets/search/?q=surface+laptop&type=link&cId=51473eaa-ad0e-4851-8c58-53d99a66dfdb&iId=53f434d1-fcac-478c-bd78-b7aca218bbdc',
'Gadget':'https://www.reddit.com/r/gadgets/search/?q=windows+laptop&type=link&cId=39e76040-e750-42f6-9e46-e3140116d199&iId=bd9a2d65-e61b-44bc-8d45-38c91adf9b23',
'Windows10':'https://www.reddit.com/r/Windows10/search/?q=surface+laptop&type=link&cId=a5220d04-fe28-4331-b688-5c344d1e1d73&iId=2f1b39aa-cf93-45d5-af1c-d2b85728ba43',
'Windows10':'https://www.reddit.com/r/Windows10/search/?q=windows+laptop&type=link&cId=cb58f88f-b2f2-4dbb-9061-d2405e37c629&iId=c9563618-80b6-42da-b34f-94b6a88063b9',
'AMD_Stock':'https://www.reddit.com/r/AMD_Stock/search/?q=surface+laptop&type=link&cId=91b302ed-7044-4045-a18d-353135fb931e&iId=6886df6c-6e9c-47d2-ac31-04331c91efa3',
'AMD_Stock':'https://www.reddit.com/r/AMD_Stock/search/?q=windows+laptop&type=link&cId=365ce7c6-e709-4d8f-a5c3-71877be104a0&iId=ac19065a-3cd3-406b-8f9c-b5b3fc9c76af',
'TechSupport':'https://www.reddit.com/r/techsupport/search/?q=surface+laptops&type=link&cId=b4c07962-630f-48fb-bea3-1891e9cd51c4&iId=e1fa70fb-d7b8-485f-8517-0ca6ea40fd95',
'TechSupport':'https://www.reddit.com/r/techsupport/search/?q=windows+laptops&type=link&cId=003e53a3-51eb-4ff5-9174-f61858b208f1&iId=bec8a539-3308-4dff-8f27-a0a99af8dc94',
'Windows':'https://www.reddit.com/r/windows/search/?q=surface+laptops&type=link&cId=a8024c3c-8f1b-4789-89c0-0cc4a9109f90&iId=9cf6aef8-a3f0-4e9d-81e2-78e25ba320bf',
'Windows':'https://www.reddit.com/r/windows/search/?q=windows+laptops&type=link&cId=abec3748-2579-4817-ba7b-f2ee77b82ed2&iId=02f05228-59c1-4b68-bc95-4bdbd6d3509d',
'Computers':'https://www.reddit.com/r/computers/search/?q=surface+laptop&type=link&cId=692a5183-c1d6-4ad0-b2ea-eac3d83b6564&iId=3d7eb002-3b22-45e9-ad5a-9337a0603c79',
'Computers':'https://www.reddit.com/r/computers/search/?q=windows+laptop&type=link&cId=a4b87360-6504-49e3-a401-5814dcd2b41f&iId=9aac317b-59a7-4752-ab90-e61efc1d74d9',
'Windows11':'https://www.reddit.com/r/Windows11/search/?q=surface+laptops&type=link&cId=8e1d51e4-5f9e-4ae1-8e7b-cd8f1f47d734&iId=ea3a7f48-e224-4810-aff5-f8250ab27f8e',
'Windows11':'https://www.reddit.com/r/Windows11/search/?q=windows+laptops&type=link&cId=33ed0264-86ea-4799-9081-8926d6a27b89&iId=d909a378-a0a0-4fbf-ac82-ab9cd850ac47',
'TechNews':'https://www.reddit.com/r/technews/search/?q=surface+laptops&type=link&cId=7da90adc-2621-4444-9c6b-5187410fb8b5&iId=d94de688-caac-400a-8c97-6feea7ee0d53',
'TechNews':'https://www.reddit.com/r/technews/search/?q=windows+laptops&type=link&cId=ad899d78-dbe1-4e7e-ab5e-754ca2a80edc&iId=3eedb7f9-4ef0-4ed1-9f8c-81902a910662'}
community = list(urls.keys())
links = []
com = []
for each in community:
    print(len(links))
    print(len(com))
    driver.quit()
    driver = webdriver.Edge(service=edge_service)
    driver.get(str(urls[each]))
    time.sleep(2)
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    try:
        box = driver.find_element('xpath','//*[@class="bg-neutral-background min-h-[calc(100vh-var(--shreddit-header-height)-var(--page-y-padding))] mx-auto xs:px-0 sm:px-md lg:px-0 "]')
        feed = box.find_element('xpath','//*[@label="search-results-page-tab-posts"]')
        all_post = feed.find_elements('xpath','//*[@class = "flex justify-between items-center p-md hover:bg-neutral-background-hover xs:rounded-[16px] my-2xs relative"]')
    except:
        pass
    for each_post in all_post:
        try:
            lk = each_post.find_element('class name','absolute.inset-0')
            z = lk.get_attribute('href')
            links.append(z)
            com.append(z+'_'+each)
        except:
            pass
        

print(len(links))
print(len(com))
links = list(set(links))
com=list(set(com))
uids = []
titles = []
dates = []
contents = []
comments =[]
u = []
it = 0
for lin in links:
    it+=1
    if it%5 == 0:
        print(it)
        driver.quit()
        driver = webdriver.Edge(service=edge_service)
    driver.get(lin)
    time.sleep(2)
    uid = lin.split('comments/')[1].split('/')[0]
    uids.append(uid)
    #to get the title of post
    try:
        title = driver.find_element('xpath','//*[@class="font-semibold text-neutral-content-strong m-0 text-18 xs:text-24  mb-xs px-md xs:px-0 xs:mb-md "]').text
    except:
        title = None
    titles.append(title)
    try:
        date = driver.find_element('xpath','//*[@class="flex gap-0 flex-col truncate"]').text.split('\n')[2]
    except:
        date = None
    try:
        date = date.replace('.','')
        date = date_con(date)
    except:
        pass
    dates.append(date)
    #to get the content of post
    try:
        driver.find_element('xpath','//*[@class="m-0 ml-[-1rem] md px-md py-0 not-italic font-normal text-sm items-center h-[28px] bg-transparent button-plain a hover:bg-transparent focus:bg-transparent active:bg-transparent visited:bg-transparent flex"]').click()
        time.sleep(1)
    except:
        pass

    try:
        rev_box = driver.find_element('xpath','//*[@class="mb-sm  mb-xs px-md xs:px-0"]')
        p = rev_box.find_elements('tag name','p')
        t = []
        for i in p:
            t.append(i.text)
        Content = '\n'.join(t)
    except:
        Content = None
    contents.append(Content)
    #to get the comments

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    while True:
        try:
            see_more = driver.find_element('xpath','//*[@class="inline-block mt-[2px] ml-px"]')
            driver.execute_script("arguments[0].scrollIntoView();", see_more)
            driver.execute_script("arguments[0].click();", see_more)
            time.sleep(2)
        except:
            break
    try:
        cm_box = driver.find_element('xpath','//*[@id="comment-tree"]')
        all_cm = cm_box.find_elements('xpath','//*[@class="md text-14 rounded-[8px] pb-2xs"]')
    except:
        cm_box = None
    if cm_box == None:
        pass
    else:
        for i in range(1,len(all_cm)):
            on = '//*[@id="comment-tree"]/shreddit-comment['
            tw = ']'
            try:
                p = driver.find_element('xpath',on+str(i)+tw)
            except:
                break
            try:
                o = replace_str(p.text)
                o = o.replace('•\n','')
                comment = o.split('ago\n')[1].split('Upvote')[0]
                comments.append(comment)
                u.append(uid)
            except:
                pass
d1 = {'uid': uids,
      'Community':com,
    'Title': titles,
     'Content':contents,
     'ReviewDate':dates}
Revs = pd.DataFrame(d1)

# d2 = {'uid': u,
#     'comments': comments}
# comments = pd.DataFrame(d2)
    
    


0
0
123
123
147
147
170
170
195
195


KeyboardInterrupt: 

In [9]:
# d1 = {
#     'uid': uids,
#       'Community':com,
#     'Title': titles,
#      'Content':contents,
#      'ReviewDate':dates}
# Revs = pd.DataFrame(d1)

# d2 = {'uid': u,
#     'comments': comments}
# comments = pd.DataFrame(d2)

In [44]:
# print(len(uids))
# print(len(com))
# print(len(titles))
# print(len(contents))
# print(len(dates))
# print(len(links))

In [43]:
# R1 = Revs.copy

In [42]:
# c1 = comments.copy

In [41]:
# ul = links.copy

In [40]:
# comments.to_csv('com123.csv')

In [34]:
# links

In [39]:
# type(d1)

In [35]:
# d1

In [37]:
# data= pd.DataFrame(d1)

In [38]:
# data.head(5)

In [32]:
# data.to_csv('reddit.csv', index=False)  